# Import Packages

In [51]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as f

spark = SparkSession.builder.appName("example").getOrCreate()

24/01/22 15:24:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# Load Data

In [102]:
input_df = pd.read_csv("./data/20240117_churn_data.csv", low_memory=False)

In [ ]:
display(input_df[input_df["welcome_discount"]])

In [104]:
display(input_df[input_df["policy_nr_hashed"] == "ypg8QA2"][["policy_nr_hashed", "year_initiation_policy_version", "welcome_discount", "welcome_discount_control_group", "d_churn", "total_premium"]])

,policy_nr_hashed,year_initiation_policy_version,welcome_discount,welcome_discount_control_group,d_churn,total_premium
84183,ypg8QA2,2021.0,0.99,WD and no LPA,0,6605.76
314198,ypg8QA2,2021.0,0.99,WD and no LPA,0,6605.76
371371,ypg8QA2,2021.0,0.99,WD and no LPA,0,6605.76


# Prepare Data

In [194]:
# Only keep customers that joined after welcome discount was introduced
input_df = input_df[input_df["year_initiation_policy"] > 2019].drop_duplicates()

input_df["years_to_churn"] = input_df["years_since_policy_started"] * input_df["d_churn_around_prolongation"]

input_df['non_relevant_churn'] = input_df.groupby('policy_nr_hashed')[['d_churn_between_prolongations', 'd_churn_cancellation']].transform('sum').sum(axis=1)

input_df = input_df[input_df['non_relevant_churn'] == 0]

# print(input_df.columns)

test = input_df.groupby("policy_nr_hashed")['welcome_discount_control_group'].apply(set).reset_index()
test["size_control"] = test["welcome_discount_control_group"].str.len()

# rLmkOem: 1 data point duplicated and not churning (no mutation)
# 0WK77VZ: discount applied for multiple years?
# A lot of those (rLmkOem, 0WK77VZ, etc.) have different values in 'welcome_discount_control_group' accross different years. We thought this should be constant per policy?

display(input_df[input_df["policy_nr_hashed"] == "JBw6R7e"][["policy_nr_hashed", "year_initiation_policy_version", "welcome_discount", "welcome_discount_control_group", "d_churn", "year_end_policy", "mutation_1"]])

# display(test[test["size_control"] == 3])

,policy_nr_hashed,year_initiation_policy_version,welcome_discount,welcome_discount_control_group,d_churn,year_end_policy,mutation_1
423572,JBw6R7e,2023.0,0.9952,WD and LPA,0,NaN,NaN


In [196]:
print((1-0.9952)*100)

0.48000000000000265
